In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.types import *
sc=SparkContext(appName="DataFrame_d2") 
spark=SparkSession.builder.appName('d2').getOrCreate()

In [3]:
staff=[(1,'mike',30,'finance',24000),(2,'lee',34,'develop',36000),(3,'allen',36,'manager',40000),(4,'jane',None,'CFO',None)] 
staff_schema = StructType([
        StructField("id", IntegerType(), True),    
        StructField("name", StringType(), True),
        StructField("age", IntegerType(), True),
        StructField("job", StringType(), True),
        StructField("salary",LongType(),True)
])
staff_df=spark.createDataFrame(staff,staff_schema)

In [4]:
user=[(1,'mike','BeiJin','朝阳'),(2,None,'ShangHai','徐汇'),(3,'allen','GuangZhou','天河'),(4,'jane','ShenZhen','福田')]
user_schema=StructType([
        StructField("id", IntegerType(), True),    
        StructField("name", StringType(), True),
        StructField("city", StringType(), True),
        StructField("region", StringType(), True)
])
user_df=spark.createDataFrame(user,user_schema)

In [7]:
cross_data=staff_df.crossJoin(user_df)

In [8]:
cross_data.show()

+---+-----+----+-------+------+---+-----+---------+------+
| id| name| age|    job|salary| id| name|     city|region|
+---+-----+----+-------+------+---+-----+---------+------+
|  1| mike|  30|finance| 24000|  1| mike|   BeiJin|æé³|
|  1| mike|  30|finance| 24000|  2| null| ShangHai|å¾æ±|
|  1| mike|  30|finance| 24000|  3|allen|GuangZhou|å¤©æ²³|
|  1| mike|  30|finance| 24000|  4| jane| ShenZhen|ç¦ç°|
|  2|  lee|  34|develop| 36000|  1| mike|   BeiJin|æé³|
|  2|  lee|  34|develop| 36000|  2| null| ShangHai|å¾æ±|
|  2|  lee|  34|develop| 36000|  3|allen|GuangZhou|å¤©æ²³|
|  2|  lee|  34|develop| 36000|  4| jane| ShenZhen|ç¦ç°|
|  3|allen|  36|manager| 40000|  1| mike|   BeiJin|æé³|
|  3|allen|  36|manager| 40000|  2| null| ShangHai|å¾æ±|
|  3|allen|  36|manager| 40000|  3|allen|GuangZhou|å¤©æ²³|
|  3|allen|  36|manager| 40000|  4| jane| ShenZhen|ç¦ç°|
|  4| jane|null|    CFO|  null|  1| mike|   BeiJin|æé³|
|  4| jane|null|    CFO|  null|  2| null| ShangHai|å¾æ±

In [12]:
except_data=user_df.select('id','name').exceptAll(staff_df.select('id','name'))

In [13]:
except_data.show()

+---+----+
| id|name|
+---+----+
|  2|null|
+---+----+



In [16]:
user_df.select('id','name').intersectAll(staff_df.select('id','name')).show()

+---+-----+
| id| name|
+---+-----+
|  4| jane|
|  3|allen|
|  1| mike|
+---+-----+



In [18]:
staff_df.join(user_df,staff_df.id==user_df.id,'left').select(staff_df.name,staff_df.salary,user_df.city).show()

+-----+------+---------+
| name|salary|     city|
+-----+------+---------+
| mike| 24000|   BeiJin|
|allen| 40000|GuangZhou|
| jane|  null| ShenZhen|
|  lee| 36000| ShangHai|
+-----+------+---------+



In [23]:
staff_df.agg({'salary':'min'}).collect()
from pyspark.sql import functions as F
staff_df.agg(F.min(staff_df.salary)).collect()

[Row(min(salary)=24000)]

In [43]:
t=staff_df.select(staff_df.age.cast(DoubleType()),staff_df.salary.cast(DoubleType()) )

In [46]:
t.corr('age','salary')

0.9714027646697837

In [47]:
staff_df.corr('age','salary')

0.9714027646697837

In [48]:
staff_df.count()

4

In [49]:
staff_df.cov('age','salary')

294666.6666666667

In [50]:
t.cov('age','salary')

294666.6666666667

In [54]:
type(staff_df.cube('age'))

pyspark.sql.group.GroupedData

In [57]:
staff_df.describe().show()

+-------+------------------+-----+------------------+-------+------------------+
|summary|                id| name|               age|    job|            salary|
+-------+------------------+-----+------------------+-------+------------------+
|  count|                 4|    4|                 3|      4|                 3|
|   mean|               2.5| null|33.333333333333336|   null|33333.333333333336|
| stddev|1.2909944487358056| null| 3.055050463303893|   null|  8326.66399786453|
|    min|                 1|allen|                30|    CFO|             24000|
|    max|                 4| mike|                36|manager|             40000|
+-------+------------------+-----+------------------+-------+------------------+



In [59]:
staff_df.groupBy().avg().collect()

[Row(avg(id)=2.5, avg(age)=33.333333333333336, avg(salary)=33333.333333333336)]

In [64]:
staff_df.describe().show()
staff_df.sort('name','salary').show()

+-------+------------------+-----+------------------+-------+------------------+
|summary|                id| name|               age|    job|            salary|
+-------+------------------+-----+------------------+-------+------------------+
|  count|                 4|    4|                 3|      4|                 3|
|   mean|               2.5| null|33.333333333333336|   null|33333.333333333336|
| stddev|1.2909944487358056| null| 3.055050463303893|   null|  8326.66399786453|
|    min|                 1|allen|                30|    CFO|             24000|
|    max|                 4| mike|                36|manager|             40000|
+-------+------------------+-----+------------------+-------+------------------+

+---+-----+----+-------+------+
| id| name| age|    job|salary|
+---+-----+----+-------+------+
|  3|allen|  36|manager| 40000|
|  4| jane|null|    CFO|  null|
|  2|  lee|  34|develop| 36000|
|  1| mike|  30|finance| 24000|
+---+-----+----+-------+------+



In [65]:
staff_df.summary("count", "min", "25%", "75%", "max").show()

+-------+---+-----+---+-------+------+
|summary| id| name|age|    job|salary|
+-------+---+-----+---+-------+------+
|  count|  4|    4|  3|      4|     3|
|    min|  1|allen| 30|    CFO| 24000|
|    25%|  1| null| 30|   null| 24000|
|    75%|  3| null| 36|   null| 40000|
|    max|  4| mike| 36|manager| 40000|
+-------+---+-----+---+-------+------+



In [66]:
staff_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- salary: long (nullable = true)



In [68]:
user_df_copy=user_df.alias('user_df_copy')

In [70]:
user_df_copy.dropna().collect()

[Row(id=1, name=u'mike', city=u'BeiJin', region=u'\xe6\x9c\x9d\xe9\x98\xb3'),
 Row(id=3, name=u'allen', city=u'GuangZhou', region=u'\xe5\xa4\xa9\xe6\xb2\xb3'),
 Row(id=4, name=u'jane', city=u'ShenZhen', region=u'\xe7\xa6\x8f\xe7\x94\xb0')]

In [71]:
user_df_copy.fillna('anonymous').collect()

[Row(id=1, name=u'mike', city=u'BeiJin', region=u'\xe6\x9c\x9d\xe9\x98\xb3'),
 Row(id=2, name=u'anonymous', city=u'ShangHai', region=u'\xe5\xbe\x90\xe6\xb1\x87'),
 Row(id=3, name=u'allen', city=u'GuangZhou', region=u'\xe5\xa4\xa9\xe6\xb2\xb3'),
 Row(id=4, name=u'jane', city=u'ShenZhen', region=u'\xe7\xa6\x8f\xe7\x94\xb0')]

In [72]:
user_df.filter(user_df.name.isNull()).collect()

[Row(id=2, name=None, city=u'ShangHai', region=u'\xe5\xbe\x90\xe6\xb1\x87')]

In [80]:
user_null=user_df.filter(user_df.name.isNull())

In [81]:
user_null.select('name','city','region')

[Row(name=None, city=u'ShangHai', region=u'\xe5\xbe\x90\xe6\xb1\x87')]

In [77]:
user_na=user_df_copy.na

In [84]:
user_na.fill('anonymous').collect()

[Row(id=1, name=u'mike', city=u'BeiJin', region=u'\xe6\x9c\x9d\xe9\x98\xb3'),
 Row(id=2, name=u'anonymous', city=u'ShangHai', region=u'\xe5\xbe\x90\xe6\xb1\x87'),
 Row(id=3, name=u'allen', city=u'GuangZhou', region=u'\xe5\xa4\xa9\xe6\xb2\xb3'),
 Row(id=4, name=u'jane', city=u'ShenZhen', region=u'\xe7\xa6\x8f\xe7\x94\xb0')]

In [88]:
user_null.explain()

== Physical Plan ==
*(1) Filter isnull(name#11)
+- Scan ExistingRDD[id#10,name#11,city#12,region#13]


In [120]:
staff_foreach=staff_df.filter("id!=4").foreach(lambda x : x.salary * 1.5 )

In [121]:
staff_foreach.show()

AttributeError: 'NoneType' object has no attribute 'show'

In [107]:
staff_df.selectExpr('name','salary *2').collect()

[Row(name=u'mike', (salary * 2)=48000),
 Row(name=u'lee', (salary * 2)=72000),
 Row(name=u'allen', (salary * 2)=80000),
 Row(name=u'jane', (salary * 2)=None)]

In [111]:
user_df.selectExpr('name',"city + '-cn' ")

DataFrame[name: string, (city + -cn): double]

In [122]:
list(user_df.toLocalIterator())

[Row(id=1, name=u'mike', city=u'BeiJin', region=u'\xe6\x9c\x9d\xe9\x98\xb3'),
 Row(id=2, name=None, city=u'ShangHai', region=u'\xe5\xbe\x90\xe6\xb1\x87'),
 Row(id=3, name=u'allen', city=u'GuangZhou', region=u'\xe5\xa4\xa9\xe6\xb2\xb3'),
 Row(id=4, name=u'jane', city=u'ShenZhen', region=u'\xe7\xa6\x8f\xe7\x94\xb0')]

In [123]:
user_df.toLocalIterator()

In [124]:
staff_df.withColumn('salary_tax',staff_df.salary * 0.05).collect()

[Row(id=1, name=u'mike', age=30, job=u'finance', salary=24000, salary_tax=1200.0),
 Row(id=2, name=u'lee', age=34, job=u'develop', salary=36000, salary_tax=1800.0),
 Row(id=3, name=u'allen', age=36, job=u'manager', salary=40000, salary_tax=2000.0),
 Row(id=4, name=u'jane', age=None, job=u'CFO', salary=None, salary_tax=None)]

In [125]:
staff_df.withColumn('salary',staff_df.salary * 1.05).collect()

[Row(id=1, name=u'mike', age=30, job=u'finance', salary=25200.0),
 Row(id=2, name=u'lee', age=34, job=u'develop', salary=37800.0),
 Row(id=3, name=u'allen', age=36, job=u'manager', salary=42000.0),
 Row(id=4, name=u'jane', age=None, job=u'CFO', salary=None)]

In [126]:
staff_df.withColumnRenamed('salary','salary_taxed').collect()

[Row(id=1, name=u'mike', age=30, job=u'finance', salary_taxed=24000),
 Row(id=2, name=u'lee', age=34, job=u'develop', salary_taxed=36000),
 Row(id=3, name=u'allen', age=36, job=u'manager', salary_taxed=40000),
 Row(id=4, name=u'jane', age=None, job=u'CFO', salary_taxed=None)]

In [127]:
for x in user_df.toLocalIterator():
    print x

Row(id=1, name=u'mike', city=u'BeiJin', region=u'\xe6\x9c\x9d\xe9\x98\xb3')
Row(id=2, name=None, city=u'ShangHai', region=u'\xe5\xbe\x90\xe6\xb1\x87')
Row(id=3, name=u'allen', city=u'GuangZhou', region=u'\xe5\xa4\xa9\xe6\xb2\xb3')
Row(id=4, name=u'jane', city=u'ShenZhen', region=u'\xe7\xa6\x8f\xe7\x94\xb0')


In [129]:
user_json=user_df.toJSON()

In [130]:
type(user_json)

pyspark.rdd.RDD

In [131]:
staff_pd=staff_df.toPandas()

In [132]:
type(staff_pd)

pandas.core.frame.DataFrame

In [135]:
staff_df.toDF('c1','c2','c3','c4','c5')

DataFrame[c1: int, c2: string, c3: int, c4: string, c5: bigint]

In [136]:
staff_df_copy=staff_df.alias('staff_copy')

In [137]:
staff_all=staff_df.union(staff_df)

In [138]:
staff_all.show()

+---+-----+----+-------+------+
| id| name| age|    job|salary|
+---+-----+----+-------+------+
|  1| mike|  30|finance| 24000|
|  2|  lee|  34|develop| 36000|
|  3|allen|  36|manager| 40000|
|  4| jane|null|    CFO|  null|
|  1| mike|  30|finance| 24000|
|  2|  lee|  34|develop| 36000|
|  3|allen|  36|manager| 40000|
|  4| jane|null|    CFO|  null|
+---+-----+----+-------+------+



In [139]:
group_name=staff_all.groupBy(staff_all.name)

In [141]:
group_name.count().head(2)

[Row(name=u'allen', count=2), Row(name=u'jane', count=2)]

In [143]:
group_name.avg().head(2)

[Row(name=u'allen', avg(id)=3.0, avg(age)=36.0, avg(salary)=40000.0),
 Row(name=u'jane', avg(id)=4.0, avg(age)=None, avg(salary)=None)]

In [144]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

In [147]:
staff_all.stat.corr('age','salary')

0.9714027646697837

In [154]:
staff_all.freqItems(['name','job']).collect()

[Row(name_freqItems=[u'mike', u'jane', u'allen', u'lee'], job_freqItems=[u'manager', u'CFO', u'finance', u'develop'])]

In [153]:
staff_all.stat.crosstab('name','salary').show()

+-----------+-----+-----+-----+----+
|name_salary|24000|36000|40000|null|
+-----------+-----+-----+-----+----+
|       mike|    2|    0|    0|   0|
|       jane|    0|    0|    0|   2|
|      allen|    0|    0|    2|   0|
|        lee|    0|    2|    0|   0|
+-----------+-----+-----+-----+----+



In [155]:
staff_all.na.drop().show()

+---+-----+---+-------+------+
| id| name|age|    job|salary|
+---+-----+---+-------+------+
|  1| mike| 30|finance| 24000|
|  2|  lee| 34|develop| 36000|
|  3|allen| 36|manager| 40000|
|  1| mike| 30|finance| 24000|
|  2|  lee| 34|develop| 36000|
|  3|allen| 36|manager| 40000|
+---+-----+---+-------+------+



In [157]:
staff_all.na.fill(-1).show()

+---+-----+---+-------+------+
| id| name|age|    job|salary|
+---+-----+---+-------+------+
|  1| mike| 30|finance| 24000|
|  2|  lee| 34|develop| 36000|
|  3|allen| 36|manager| 40000|
|  4| jane| -1|    CFO|    -1|
|  1| mike| 30|finance| 24000|
|  2|  lee| 34|develop| 36000|
|  3|allen| 36|manager| 40000|
|  4| jane| -1|    CFO|    -1|
+---+-----+---+-------+------+



In [160]:
df = spark.createDataFrame([('2019-10-31','Thur')], ['date','week'])

In [161]:
df.show()

+----------+----+
|      date|week|
+----------+----+
|2019-10-31|Thur|
+----------+----+

